# データ数、データを可視化して分析  
コンテストページからダウンロードしたデータがどのようなデータであるのか、またどのようなデータで数はどの程度あるのかを確認します。  
これにより、以下を行います。
* 画像を見て、その画像が欠陥である確率値を予測  
* 学習の工夫に対する思考  

    
  

### ライブラリのインポート

In [1]:
from google.colab import drive
import os
import math
import random
from glob import glob
import cv2
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# GoogleDriveのマウント
drive.mount('/content/drive')
WorkDir="/content/drive/MyDrive/industry_defect_detection"
%cd {WorkDir}

データ数の確認

In [ ]:
df_train = pd.read_table("train_annotations.tsv", index_col=0)
df_train_normal = df_train[df_train.label == 0]  # 正常画像
df_train_abnormal = df_train[df_train.label == 1]  # 異常画像
print(f"正常画像: {len(df_train_normal)}, 異常画像: {len(df_train_abnormal)}, 合計: {len(df_train)}")
images_test = sorted(glob("test/*.png"))
print(f"テスト画像: {len(images_test)}")
print(f"学習画像: {len(sorted(glob('train/*.png')))}")

データの分布をヒストグラムを使用して可視化

In [ ]:
fig = plt.figure(figsize=(12, 4))

# 正常画像のヒストグラム（パターン）
ax = fig.add_subplot(1, 2, 1)
df_train_normal.pattern.hist()
ax.set_title(f"Train: Normal")

# 異常画像のヒストグラム（パターン）
ax = fig.add_subplot(1, 2, 2)
df_train_abnormal.pattern.hist()
ax.set_title(f"Train: Abnormal")

データの確認  
※ 画像の有無がFalseになっている場合、画像データの解凍(UNZIP)後GoogleDriveとの同期がうまく行っていない場合があります。  
画像データの解凍のからやり直しをお勧めします。  
『 MlCompe2023_データアップロード_データ準備.ipynb』のファイルアップロード以外をすべて実行。  
同期が完了するまで、画面を閉じずに放置(30分程度)

In [ ]:
fig = plt.figure(figsize=(12, 4))

# 正常画像の可視化
sampled_normal = df_train_normal.sample().iloc[0]
image_path = WorkDir + f"/train/{sampled_normal.name}.png"
print("正常画像サンプル： " + image_path)
print("画像の有無： " + str(os.path.exists(image_path)))
image_normal = cv2.imread(image_path)

ax = fig.add_subplot(1, 3, 1)
plt.imshow(image_normal)
ax.set_title(f"Train: Normal (Pattern {sampled_normal.pattern})")

# 異常画像の可視化
sampled_abnormal = df_train_abnormal.sample().iloc[0]
image_path = WorkDir + f"/train/{sampled_abnormal.name}.png"
print("異常画像サンプル： " + image_path)
print("画像の有無： " + str(os.path.exists(image_path)))
image_abnormal = cv2.imread(image_path)
box = (
    [
        float(sampled_abnormal.x_position),
        float(sampled_abnormal.y_position)
    ],
    [
        round(float(sampled_abnormal.semi_major_axis) * 2),
        round(float(sampled_abnormal.semi_minor_axis) * 2)
    ],
    math.degrees(float(sampled_abnormal.rotation_angle))
)
image_abnormal = cv2.ellipse(image_abnormal, box, color=(0, 0, 255), thickness=2, lineType=cv2.LINE_8)
ax = fig.add_subplot(1, 3, 2)
plt.imshow(image_abnormal)
ax.set_title(f"Train: Abnormal (Pattern {sampled_abnormal.pattern})")

# テスト画像の可視化
image_test = cv2.imread(random.choice(images_test))
ax = fig.add_subplot(1, 3, 3)
plt.imshow(image_test)
ax.set_title("Test")



学習を安定的に行うために、学習で使用する画像の画素値の平均と標準偏差を求めておきます。  
画像の正規化時に演算結果を使用します。

In [ ]:
# 画素値の平均と標準偏差を求める

paths = sorted(df_train.index.tolist())

psum = np.zeros((3, ), dtype=np.float32)
psum_sq = np.zeros((3, ), dtype=np.float32)
count = 0
for path in tqdm(paths):
    img = cv2.imread(f"train/{path}.png").astype(np.float32)[..., ::-1]
    psum += img.sum(axis=0).sum(axis=0)
    psum_sq += (img ** 2).sum(axis=0).sum(axis=0)
    count += img.shape[0] * img.shape[1]

total_mean = psum / count
total_var  = (psum_sq / count) - (total_mean ** 2)
total_std  = np.sqrt(total_var)

print(f"mean: {total_mean}, std: {total_std}")


データ数、データの分布を可視化、データの確認をすることで、扱うデータがどのようなものなのかが分かります。  

* 学習用画像には、以下の教師データが存在。

  * 正常・欠陥に共通

    * 欠陥である確率 (0, 1 の2値)

    * 画像パターン (A, B, C, D の4パターン)

  * 欠陥画像のみ、欠陥箇所（楕円表示）に関する教師データが存在

    * semi_major_axis: 欠陥の半長軸

    * semi_minor_axis: 欠陥の半短軸

    * rotation_angle: 欠陥の回転角（ラジアン）

    * x_position: 欠陥の楕円体の中心のx座標

    * y_position: 欠陥の楕円体の中心のy座標